In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 185, 185

top_model_weights_path = 'c:\Downloads\vgg16_weights.h5'
train_data_dir = 'Posters 660/Train'
validation_data_dir = 'Posters 660/Validation'
nb_train_samples = 396
nb_validation_samples = 264
epochs = 50
batch_size = 12 #modified so easily divides 396 and 264


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'w'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'w'),
            bottleneck_features_validation)

save_bottlebeck_features()

Found 396 images belonging to 3 classes.
Found 264 images belonging to 3 classes.


In [6]:
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy'))
    train_labels = np.array(
        [0] * (nb_train_samples / 2) + [1] * (nb_train_samples / 2))

    validation_data = np.load(open('bottleneck_features_validation.npy'))
    validation_labels = np.array(
        [0] * (nb_validation_samples / 2) + [1] * (nb_validation_samples / 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)

train_top_model()

Train on 396 samples, validate on 264 samples
Epoch 1/50
396/396 [==============================] - 9s - loss: 2.8786 - acc: 0.5455 - val_loss: 1.2416 - val_acc: 0.5795
Epoch 2/50
396/396 [==============================] - 4s - loss: 1.0119 - acc: 0.6414 - val_loss: 0.9460 - val_acc: 0.6174
Epoch 3/50
396/396 [==============================] - 3s - loss: 0.6080 - acc: 0.7702 - val_loss: 0.9607 - val_acc: 0.5909
Epoch 4/50
396/396 [==============================] - 3s - loss: 0.6499 - acc: 0.7677 - val_loss: 0.8851 - val_acc: 0.6326
Epoch 5/50
396/396 [==============================] - 2s - loss: 0.4780 - acc: 0.8005 - val_loss: 0.8980 - val_acc: 0.6326
Epoch 6/50
396/396 [==============================] - 3s - loss: 0.3955 - acc: 0.8460 - val_loss: 0.7714 - val_acc: 0.6780
Epoch 7/50
396/396 [==============================] - 3s - loss: 0.3319 - acc: 0.8409 - val_loss: 0.8376 - val_acc: 0.6742
Epoch 8/50
396/396 [==============================] - 2s - loss: 0.3769 - acc: 0.8561 - val_l

In [7]:
train_data = np.load(open('bottleneck_features_train.npy'))
# the features were saved in order, so recreating the labels is easy
train_labels = np.array([0] * 198 + [1] * 198) #change number of labels here

validation_data = np.load(open('bottleneck_features_validation.npy'))
validation_labels = np.array([0] * 132 + [1] * 132)

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=50,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))
model.save_weights('bottleneck_fc_model.h5')

Train on 396 samples, validate on 264 samples
Epoch 1/50
396/396 [==============================] - 3s - loss: 3.0455 - acc: 0.5227 - val_loss: 0.6354 - val_acc: 0.6629
Epoch 2/50
396/396 [==============================] - 2s - loss: 0.9064 - acc: 0.6515 - val_loss: 1.0160 - val_acc: 0.5417
Epoch 3/50
396/396 [==============================] - 2s - loss: 0.5891 - acc: 0.7424 - val_loss: 0.6081 - val_acc: 0.6780
Epoch 4/50
396/396 [==============================] - 2s - loss: 0.5345 - acc: 0.7626 - val_loss: 0.9262 - val_acc: 0.6364
Epoch 5/50
396/396 [==============================] - 2s - loss: 0.4576 - acc: 0.8056 - val_loss: 0.9383 - val_acc: 0.6212
Epoch 6/50
396/396 [==============================] - 2s - loss: 0.4008 - acc: 0.8258 - val_loss: 0.9852 - val_acc: 0.6174
Epoch 7/50
396/396 [==============================] - 2s - loss: 0.3453 - acc: 0.8763 - val_loss: 1.0438 - val_acc: 0.6439
Epoch 8/50
396/396 [==============================] - 2s - loss: 0.2748 - acc: 0.8864 - val_l